In [1]:
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise
import matplotlib.pyplot as plt
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3 import PPO,DDPG,DQN
from sb3_contrib import QRDQN
from stable_baselines3.common.buffers import PrioritizedReplayBuffer
from stable_baselines3.common.vec_env.dummy_vec_env import DummyVecEnv
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize, SubprocVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.callbacks import BaseCallback, EvalCallback
from stable_baselines3.common.evaluation import evaluate_policy
import gym
import numpy as np
import time
from stable_baselines3.common.callbacks import EvalCallback
import env
env = env.ElectricVehicleEnv()
#vec_env = make_vec_env(env, n_envs=4)
check_env(env)
env = DummyVecEnv([lambda: env])

ImportError: cannot import name 'PrioritizedReplayBuffer' from 'stable_baselines3.common.buffers' (C:\ProgramData\anaconda3\envs\UAV\lib\site-packages\stable_baselines3\common\buffers.py)

In [ ]:
# 设置评估频率
eval_freq = 1000  # 每训练500步评估一次

# 配置EvalCallback
eval_callback = EvalCallback(
    env,
    callback_on_new_best=None,
    n_eval_episodes=20,
    best_model_save_path="./best_model/DQN",  # 更改保存路径为DQN
    log_path="./logs/DQN",  # 更改日志路径为DQN
    eval_freq=eval_freq,
)

# QRDQN的超参数设置，包括优先经验回放
model = QRDQN(
    'MlpPolicy', 
    env, 
    verbose=1, 
    tensorboard_log="./logs/QRDQN",
    learning_rate=0.0005,  # 默认学习率，通常比A2C更小
    buffer_size=100000,  # 经验回放缓冲区大小
    learning_starts=1000,  # 开始学习前的步数
    batch_size=32,  # 批量大小
    tau=1.0,  # 目标网络软更新的系数
    gamma=0.99,  # 折扣因子
    train_freq=4,  # 训练的频率（每4步训练一次）
    target_update_interval=1000,  # 目标网络更新的频率
    exploration_fraction=0.1,  # 探索率衰减部分
    exploration_final_eps=0.02,  # 最小探索率
    max_grad_norm=10,  # 梯度裁剪的最大范数
    replay_buffer_class=PrioritizedReplayBuffer,  # 使用优先经验回放缓冲区
    replay_buffer_kwargs=dict(
        alpha=0.6,  # 优先经验回放的 alpha 参数
        beta=0.4,   # 优先经验回放的 beta 参数
        beta_schedule="linear",  # beta 的调度，可以是"linear"或其他策略
    ),
)


# 训练模型，并将TensorBoard日志存储到log目录中
model.learn(total_timesteps=1000000, tb_log_name="QRDQN", callback=eval_callback)

# 保存模型
model.save("qrdqn_electric_vehicle")

# 加载模型（如果需要）
# model = DQN.load("dqn_electric_vehicle")

# 评估模型
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10)
print(f"Mean reward: {mean_reward}, Std reward: {std_reward}")

# 测试训练好的模型
obs = env.reset()
for i in range(100):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    env.render()
    if done:
        obs = env.reset()

Eval num_timesteps=1000, episode_reward=-7460.05 +/- 2466.43
Episode length: 2.60 +/- 1.07
New best mean reward!
Eval num_timesteps=2000, episode_reward=-7755.41 +/- 1767.47
Episode length: 2.10 +/- 0.44
Eval num_timesteps=3000, episode_reward=-7780.50 +/- 1808.84
Episode length: 1.95 +/- 0.22
Eval num_timesteps=4000, episode_reward=-7355.77 +/- 2486.45
Episode length: 1.90 +/- 0.30
New best mean reward!
Eval num_timesteps=5000, episode_reward=-7330.89 +/- 2529.30
Episode length: 2.15 +/- 0.65
New best mean reward!
Eval num_timesteps=6000, episode_reward=-7761.44 +/- 1746.15
Episode length: 2.05 +/- 0.50
Eval num_timesteps=7000, episode_reward=-8193.78 +/- 118.78
Episode length: 2.40 +/- 0.80
Eval num_timesteps=8000, episode_reward=-8337.03 +/- 140.94
Episode length: 3.20 +/- 0.93
Eval num_timesteps=9000, episode_reward=-8233.52 +/- 148.48
Episode length: 2.60 +/- 0.97
Eval num_timesteps=10000, episode_reward=-7791.94 +/- 1774.39
Episode length: 1.95 +/- 0.22
Eval num_timesteps=11000, 